In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import regex as re
pd.set_option("display.max_columns", None)

df = pd.read_csv('2019-airline-delays-and-cancellations/flights_sample.csv')
df.shape

(1186213, 40)

In [4]:
from xgboost import XGBClassifier
import pickle

clf = pickle.load(open('xgb_clf_pkl', 'rb'))

In [13]:
import regex as re

def encode(df):
    dep_airport_ohe = pd.get_dummies(df['DEP_AIRPORT_ID'])
    arr_airport_ohe = pd.get_dummies(df['ARR_AIRPORT_ID'])
    dep_airport_ohe = dep_airport_ohe.rename(columns=lambda x: re.sub("^", "DEP_AIRPORT_", str(x)))
    arr_airport_ohe = arr_airport_ohe.rename(columns=lambda x: re.sub("^", "ARR_AIRPORT_", str(x)))
    df = df.drop(['MONTH','CARRIER_NAME', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'AIRLINE_ID'], axis=1)
    df = pd.concat([df, dep_airport_ohe], axis=1)
    df = pd.concat([df, arr_airport_ohe], axis=1)
    
    dep_time_block = pd.get_dummies(df['DEP_TIME_BLK'])
    arr_time_block = pd.get_dummies(df['ARR_TIME_BLK'])
    dep_time_block = dep_time_block.rename(columns=lambda x: re.sub("^", "DEP_TIME_", str(x)))
    arr_time_block = arr_time_block.rename(columns=lambda x: re.sub("^", "ARR_TIME_", str(x)))
    df = pd.concat([df, dep_time_block], axis=1)
    df = pd.concat([df, arr_time_block], axis=1)
    df.drop(['DEP_AIRPORT_ID', 'ARR_AIRPORT_ID', 'DEP_TIME', 'DEP_TIME_BLK', 
             'ARR_TIME', 'ARR_TIME_BLK'], axis=1, inplace=True)
    
    return df


In [14]:
df = encode(df)

In [15]:
X = df.drop(['ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_TIME_Nan'], axis=1)

In [16]:
x_test = X[:100]
clf.predict(x_test)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [18]:
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_AIRPORT_ID,ARR_AIRPORT_ID,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,AIRLINE_ID,CARRIER_NAME,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP,PRCP_ARR,SNOW_ARR,SNWD_ARR,TMAX_ARR,TMIN_ARR,WDF2_ARR,WSF2_ARR,AWND_ARR,ARR_DEL15,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,ARR_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST,ARR_BLOCK_HIST
0,9,30,1,11057,11292,1615.0,0.0,0.0,1600-1659,1737.0,0.0,1700-1759,1337.0,6,4.0,19977,United Air Lines Inc.,0.00,0.0,0.0,91.0,70.0,70.0,12.1,5.82,0.00,0.0,0.0,76.0,50.0,30.0,25.1,11.41,0,0.167208,0.138449,0.143890,0.135100,0.179688,0.156900
1,8,30,5,13495,10821,1321.0,6.0,0.0,1300-1359,1640.0,0.0,1600-1659,998.0,4,3.0,19393,Southwest Airlines Co.,0.00,0.0,0.0,95.0,75.0,30.0,13.0,4.70,0.00,0.0,0.0,91.0,58.0,270.0,14.1,4.70,0,0.192353,0.163102,0.183021,0.212659,0.197496,0.179450
2,3,13,3,11057,11618,1309.0,84.0,1.0,1100-1159,1457.0,67.0,1400-1459,529.0,3,2.0,19805,American Airlines Inc.,0.00,0.0,0.0,66.0,34.0,170.0,14.1,5.59,0.00,0.0,0.0,47.0,28.0,130.0,14.1,5.14,1,0.189410,0.179731,0.265083,0.152191,0.165468,0.152607
3,12,13,5,11057,10257,2210.0,1.0,0.0,2200-2259,2350.0,0.0,2300-2359,646.0,3,2.0,19805,American Airlines Inc.,1.73,0.0,0.0,43.0,34.0,50.0,16.1,8.28,0.04,0.0,0.0,40.0,30.0,160.0,18.1,6.26,0,0.176659,0.182110,0.244121,0.190847,0.289847,0.304861
4,3,7,4,11057,10994,745.0,0.0,0.0,0700-0759,853.0,0.0,0800-0859,168.0,1,2.0,20417,Comair Inc.,0.00,0.0,0.0,54.0,28.0,120.0,13.0,5.37,0.00,0.0,0.0,63.0,31.0,170.0,15.0,3.80,0,0.171451,0.179731,0.170961,0.179561,0.086026,0.060502


In [26]:
import datetime

weekdays = []
for i in range(100):
    if df.iloc[i].MONTH <= 2:
        x_date = datetime.date(2020, df.iloc[i].MONTH, df.iloc[i].DAY_OF_MONTH)
    else:
        x_date = datetime.date(2019, df.iloc[i].MONTH, df.iloc[i].DAY_OF_MONTH)
    weekdays.append(x_date.weekday()+1)
    
print(weekdays)

[1, 5, 3, 5, 4, 3, 6, 7, 5, 2, 2, 5, 7, 2, 6, 1, 2, 3, 1, 5, 7, 7, 6, 5, 7, 2, 6, 2, 6, 6, 5, 2, 1, 2, 5, 7, 3, 3, 4, 2, 1, 1, 2, 7, 7, 2, 5, 2, 4, 5, 2, 7, 4, 3, 7, 7, 1, 5, 5, 1, 2, 6, 2, 1, 7, 7, 2, 4, 4, 5, 1, 4, 4, 1, 5, 4, 2, 7, 6, 3, 1, 3, 4, 5, 5, 6, 1, 1, 2, 2, 6, 5, 1, 1, 2, 4, 3, 7, 2, 3]


In [34]:

time = df.iloc[36611].DEP_TIME
print(int(time / 100))
a = 22
dep_time_block = "0000-0559"
str_a = str(a)
dep_time_block = re.sub("^..", str(a), dep_time_block)
dep_time_block = re.sub("(?<=-)..", str(a), dep_time_block)
dep_time_block

18


'2200-2259'

In [35]:
def find_time_block(time):
    res = "0000-0559"
    hour = int(time / 100)
    if hour < 6:
        return res
    else:
        res = re.sub("^..", str(hour), res)
        res = re.sub("(?<=-)..", str(hour), res)
    return res

In [40]:
find_time_block(df.iloc[880769].DEP_TIME)

'600-659'

In [27]:
len(df.DEP_AIRPORT_ID.unique())

96

In [35]:
airports = pd.read_csv('2019-airline-delays-and-cancellations/raw_data/airports_list.csv')
airports.drop(['DISPLAY_AIRPORT_NAME', 'ORIGIN_CITY_NAME', 'NAME'], axis=1, inplace=True)
airports.head()

,ORIGIN_AIRPORT_ID
0,12992
1,10257
2,10140
3,10299
4,10397


In [36]:
coords = pd.read_csv('2019-airline-delays-and-cancellations/raw_data/AIRPORT_COORDINATES.csv')
coords.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,10001,Afognak Lake Airport,58.109444,-152.906667
1,10003,Bear Creek Mining Strip,65.548056,-161.071667
2,10004,Lik Mining Camp,68.083333,-163.166667
3,10005,Little Squaw Airport,67.570000,-148.183889
4,10006,Kizhuyak Bay,57.745278,-152.882778


In [37]:
airports = airports.merge(coords, how='inner', on='ORIGIN_AIRPORT_ID')
airports.head(10)

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,12992,Adams Field,34.729167,-92.234444
1,12992,Adams Field,34.728889,-92.224722
2,12992,Adams Field,34.729444,-92.224722
3,12992,Bill and Hillary Clinton Nat Adams Field,34.729444,-92.224722
4,12992,Adams Field,34.729444,-92.224722
5,12992,Bill and Hillary Clinton Nat Adams Field,34.729444,-92.224722
6,10257,Albany International,42.745833,-73.805278
7,10257,Albany International,42.749167,-73.801944
8,10140,Albuquerque International Sunport,35.041667,-106.606389
9,10140,Albuquerque International Sunport,35.040278,-106.609167


In [38]:
airports = airports.drop_duplicates(subset=['ORIGIN_AIRPORT_ID'], keep='first')
airports.head(10)

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,12992,Adams Field,34.729167,-92.234444
6,10257,Albany International,42.745833,-73.805278
8,10140,Albuquerque International Sunport,35.041667,-106.606389
13,10299,Anchorage International,61.169167,-149.985278
19,10397,Atlanta Municipal,33.640833,-84.427222
26,10423,Austin - Bergstrom International,30.194444,-97.670000
28,10599,Birmingham Airport,33.563056,-86.755556
32,10529,Bradley International,41.938611,-72.683333
38,10994,Charleston International,32.899167,-80.038889
40,13232,Chicago Midway International,41.785000,-87.751944


In [20]:
len(airports)

97

In [41]:
airports.to_csv('2019-airline-delays-and-cancellations/airports_table.csv', index=False)

In [40]:
airports.rename(columns={'ORIGIN_AIRPORT_ID': 'ID', 'DISPLAY_AIRPORT_NAME': 'NAME'}, inplace=True)

In [24]:
airports.head()

,ID,NAME,LATITUDE,LONGITUDE
0,12992,Adams Field,34.729167,-92.234444
6,10257,Albany International,42.745833,-73.805278
8,10140,Albuquerque International Sunport,35.041667,-106.606389
13,10299,Anchorage International,61.169167,-149.985278
19,10397,Atlanta Municipal,33.640833,-84.427222


In [39]:
airports.loc[airports.DISPLAY_AIRPORT_NAME == 'Honolulu International']

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
123,12173,Honolulu International,21.318611,-157.9225


In [3]:
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_AIRPORT_ID,ARR_AIRPORT_ID,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,AIRLINE_ID,CARRIER_NAME,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP,PRCP_ARR,SNOW_ARR,SNWD_ARR,TMAX_ARR,TMIN_ARR,WDF2_ARR,WSF2_ARR,AWND_ARR,ARR_DEL15,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,ARR_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST,ARR_BLOCK_HIST
0,9,30,1,11057,11292,1615.0,0.0,0.0,1600-1659,1737.0,0.0,1700-1759,1337.0,6,4.0,19977,United Air Lines Inc.,0.00,0.0,0.0,91.0,70.0,70.0,12.1,5.82,0.00,0.0,0.0,76.0,50.0,30.0,25.1,11.41,0,0.167208,0.138449,0.143890,0.135100,0.179688,0.156900
1,8,30,5,13495,10821,1321.0,6.0,0.0,1300-1359,1640.0,0.0,1600-1659,998.0,4,3.0,19393,Southwest Airlines Co.,0.00,0.0,0.0,95.0,75.0,30.0,13.0,4.70,0.00,0.0,0.0,91.0,58.0,270.0,14.1,4.70,0,0.192353,0.163102,0.183021,0.212659,0.197496,0.179450
2,3,13,3,11057,11618,1309.0,84.0,1.0,1100-1159,1457.0,67.0,1400-1459,529.0,3,2.0,19805,American Airlines Inc.,0.00,0.0,0.0,66.0,34.0,170.0,14.1,5.59,0.00,0.0,0.0,47.0,28.0,130.0,14.1,5.14,1,0.189410,0.179731,0.265083,0.152191,0.165468,0.152607
3,12,13,5,11057,10257,2210.0,1.0,0.0,2200-2259,2350.0,0.0,2300-2359,646.0,3,2.0,19805,American Airlines Inc.,1.73,0.0,0.0,43.0,34.0,50.0,16.1,8.28,0.04,0.0,0.0,40.0,30.0,160.0,18.1,6.26,0,0.176659,0.182110,0.244121,0.190847,0.289847,0.304861
4,3,7,4,11057,10994,745.0,0.0,0.0,0700-0759,853.0,0.0,0800-0859,168.0,1,2.0,20417,Comair Inc.,0.00,0.0,0.0,54.0,28.0,120.0,13.0,5.37,0.00,0.0,0.0,63.0,31.0,170.0,15.0,3.80,0,0.171451,0.179731,0.170961,0.179561,0.086026,0.060502


In [12]:
df.groupby(by=['MONTH', 'DAY_OF_MONTH', 'DEP_AIRPORT_ID']).head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_AIRPORT_ID,ARR_AIRPORT_ID,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,AIRLINE_ID,CARRIER_NAME,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP,PRCP_ARR,SNOW_ARR,SNWD_ARR,TMAX_ARR,TMIN_ARR,WDF2_ARR,WSF2_ARR,AWND_ARR,ARR_DEL15,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,ARR_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST,ARR_BLOCK_HIST
0,9,30,1,11057,11292,1615.0,0.0,0.0,1600-1659,1737.0,0.0,1700-1759,1337.0,6,4.0,19977,United Air Lines Inc.,0.00,0.0,0.0,91.0,70.0,70.0,12.1,5.82,0.00,0.0,0.0,76.0,50.0,30.0,25.1,11.41,0,0.167208,0.138449,0.143890,0.135100,0.179688,0.156900
1,8,30,5,13495,10821,1321.0,6.0,0.0,1300-1359,1640.0,0.0,1600-1659,998.0,4,3.0,19393,Southwest Airlines Co.,0.00,0.0,0.0,95.0,75.0,30.0,13.0,4.70,0.00,0.0,0.0,91.0,58.0,270.0,14.1,4.70,0,0.192353,0.163102,0.183021,0.212659,0.197496,0.179450
2,3,13,3,11057,11618,1309.0,84.0,1.0,1100-1159,1457.0,67.0,1400-1459,529.0,3,2.0,19805,American Airlines Inc.,0.00,0.0,0.0,66.0,34.0,170.0,14.1,5.59,0.00,0.0,0.0,47.0,28.0,130.0,14.1,5.14,1,0.189410,0.179731,0.265083,0.152191,0.165468,0.152607
3,12,13,5,11057,10257,2210.0,1.0,0.0,2200-2259,2350.0,0.0,2300-2359,646.0,3,2.0,19805,American Airlines Inc.,1.73,0.0,0.0,43.0,34.0,50.0,16.1,8.28,0.04,0.0,0.0,40.0,30.0,160.0,18.1,6.26,0,0.176659,0.182110,0.244121,0.190847,0.289847,0.304861
4,3,7,4,11057,10994,745.0,0.0,0.0,0700-0759,853.0,0.0,0800-0859,168.0,1,2.0,20417,Comair Inc.,0.00,0.0,0.0,54.0,28.0,120.0,13.0,5.37,0.00,0.0,0.0,63.0,31.0,170.0,15.0,3.80,0,0.171451,0.179731,0.170961,0.179561,0.086026,0.060502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185905,1,19,7,15412,12953,1327.0,0.0,0.0,1300-1359,1520.0,0.0,1500-1559,648.0,3,1.0,20363,Endeavor Air Inc.,1.58,0.0,0.0,57.0,40.0,240.0,25.9,8.28,0.34,0.0,0.0,39.0,32.0,70.0,16.1,9.40,0,0.113517,0.141321,0.162440,0.141395,0.142004,0.124584
1186023,12,29,7,11638,14107,1624.0,0.0,0.0,1600-1659,1904.0,0.0,1900-1959,493.0,2,5.0,20378,Mesa Airlines Inc.,0.01,0.0,0.0,54.0,39.0,130.0,10.1,3.13,0.00,0.0,0.0,53.0,35.0,240.0,12.1,3.58,0,0.215049,0.183333,0.205912,0.223586,0.253337,0.254017
1186148,12,5,4,14761,14122,1058.0,0.0,0.0,1000-1059,1322.0,14.0,1300-1359,809.0,4,2.0,20368,Allegiant Air,0.00,0.0,0.0,70.0,33.0,340.0,8.9,1.12,0.00,0.0,0.0,39.0,31.0,280.0,18.1,9.62,0,0.264289,0.213415,0.201385,0.182013,0.183004,0.156418
1186197,11,28,4,15370,12266,915.0,0.0,0.0,0900-0959,1121.0,27.0,1100-1159,429.0,2,1.0,20366,Atlantic Southeast Airlines,0.17,0.0,0.0,41.0,36.0,90.0,16.1,9.17,0.00,0.0,0.0,88.0,75.0,60.0,16.1,7.83,1,0.180221,0.105900,0.130479,0.141108,0.114738,0.104582


In [52]:
weather = df.drop_duplicates(subset=['MONTH', 'DAY_OF_MONTH', 'DEP_AIRPORT_ID']).sort_values( \
                                    by=['MONTH', 'DAY_OF_MONTH', 'DEP_AIRPORT_ID'])
weather = weather[['MONTH', 'DAY_OF_MONTH', 'DEP_AIRPORT_ID', 'PRCP_DEP', 'SNOW_DEP', 'SNWD_DEP', 'TMAX_DEP', 'TMIN_DEP', 'WDF2_DEP', 'WSF2_DEP', 'AWND_DEP']]
weather.rename(columns={'DEP_AIRPORT_ID': 'AIRPORT_ID'}, inplace=True)
weather.head()

,MONTH,DAY_OF_MONTH,AIRPORT_ID,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP
53413,1,1,10140,0.06,2.0,2.0,34.0,15.0,110.0,38.0,18.34
38130,1,1,10257,0.10,0.0,0.0,53.0,27.0,280.0,36.0,15.21
108240,1,1,10299,0.03,0.2,13.0,42.0,31.0,280.0,30.0,12.75
4833,1,1,10397,0.14,0.0,0.0,66.0,57.0,310.0,15.0,4.70
128754,1,1,10423,0.00,0.0,0.0,46.0,33.0,20.0,23.0,12.30


In [4]:
weather = pd.read_csv('2019-airline-delays-and-cancellations/weather_table.csv')
weather.head()

,MONTH,DAY_OF_MONTH,AIRPORT_ID,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP
0,1,1,10140,0.06,2.0,2.0,34.0,15.0,110.0,38.0,18.34
1,1,1,10257,0.10,0.0,0.0,53.0,27.0,280.0,36.0,15.21
2,1,1,10299,0.03,0.2,13.0,42.0,31.0,280.0,30.0,12.75
3,1,1,10397,0.14,0.0,0.0,66.0,57.0,310.0,15.0,4.70
4,1,1,10423,0.00,0.0,0.0,46.0,33.0,20.0,23.0,12.30


In [7]:
weather = weather.rename(columns=lambda x: re.sub('_DEP$','',x))
weather.head()

,MONTH,DAY_OF_MONTH,AIRPORT_ID,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WSF2,AWND
0,1,1,10140,0.06,2.0,2.0,34.0,15.0,110.0,38.0,18.34
1,1,1,10257,0.10,0.0,0.0,53.0,27.0,280.0,36.0,15.21
2,1,1,10299,0.03,0.2,13.0,42.0,31.0,280.0,30.0,12.75
3,1,1,10397,0.14,0.0,0.0,66.0,57.0,310.0,15.0,4.70
4,1,1,10423,0.00,0.0,0.0,46.0,33.0,20.0,23.0,12.30


In [8]:
weather.to_csv('2019-airline-delays-and-cancellations/weather_table.csv', index=False)

In [12]:
df.AIRLINE_ID.unique()

array([19977, 19393, 19805, 20417, 20378, 19930, 20366, 20304, 20363,
       20314, 20436, 19790, 20416, 20398, 19690, 20409, 20368],
      dtype=int64)

In [9]:
df.CARRIER_NAME.unique()

array(['United Air Lines Inc.', 'Southwest Airlines Co.',
       'American Airlines Inc.', 'Comair Inc.', 'Mesa Airlines Inc.',
       'Alaska Airlines Inc.', 'Atlantic Southeast Airlines',
       'SkyWest Airlines Inc.', 'Endeavor Air Inc.',
       'Midwest Airline, Inc.', 'Frontier Airlines Inc.',
       'Delta Air Lines Inc.', 'Spirit Air Lines',
       'American Eagle Airlines Inc.', 'Hawaiian Airlines Inc.',
       'JetBlue Airways', 'Allegiant Air'], dtype=object)

In [10]:
len(df.CARRIER_NAME.unique())

17

In [11]:
len(df.AIRLINE_ID.unique())

17

In [3]:
df.rename(columns={'CARRIER_NAME': 'AIRLINE_NAME'}, inplace=True)
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEP_AIRPORT_ID,ARR_AIRPORT_ID,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,DISTANCE,DISTANCE_GROUP,SEGMENT_NUMBER,AIRLINE_ID,AIRLINE_NAME,PRCP_DEP,SNOW_DEP,SNWD_DEP,TMAX_DEP,TMIN_DEP,WDF2_DEP,WSF2_DEP,AWND_DEP,PRCP_ARR,SNOW_ARR,SNWD_ARR,TMAX_ARR,TMIN_ARR,WDF2_ARR,WSF2_ARR,AWND_ARR,ARR_DEL15,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,ARR_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST,ARR_BLOCK_HIST
0,9,30,1,11057,11292,1615.0,0.0,0.0,1600-1659,1737.0,0.0,1700-1759,1337.0,6,4.0,19977,United Air Lines Inc.,0.00,0.0,0.0,91.0,70.0,70.0,12.1,5.82,0.00,0.0,0.0,76.0,50.0,30.0,25.1,11.41,0,0.167208,0.138449,0.143890,0.135100,0.179688,0.156900
1,8,30,5,13495,10821,1321.0,6.0,0.0,1300-1359,1640.0,0.0,1600-1659,998.0,4,3.0,19393,Southwest Airlines Co.,0.00,0.0,0.0,95.0,75.0,30.0,13.0,4.70,0.00,0.0,0.0,91.0,58.0,270.0,14.1,4.70,0,0.192353,0.163102,0.183021,0.212659,0.197496,0.179450
2,3,13,3,11057,11618,1309.0,84.0,1.0,1100-1159,1457.0,67.0,1400-1459,529.0,3,2.0,19805,American Airlines Inc.,0.00,0.0,0.0,66.0,34.0,170.0,14.1,5.59,0.00,0.0,0.0,47.0,28.0,130.0,14.1,5.14,1,0.189410,0.179731,0.265083,0.152191,0.165468,0.152607
3,12,13,5,11057,10257,2210.0,1.0,0.0,2200-2259,2350.0,0.0,2300-2359,646.0,3,2.0,19805,American Airlines Inc.,1.73,0.0,0.0,43.0,34.0,50.0,16.1,8.28,0.04,0.0,0.0,40.0,30.0,160.0,18.1,6.26,0,0.176659,0.182110,0.244121,0.190847,0.289847,0.304861
4,3,7,4,11057,10994,745.0,0.0,0.0,0700-0759,853.0,0.0,0800-0859,168.0,1,2.0,20417,Comair Inc.,0.00,0.0,0.0,54.0,28.0,120.0,13.0,5.37,0.00,0.0,0.0,63.0,31.0,170.0,15.0,3.80,0,0.171451,0.179731,0.170961,0.179561,0.086026,0.060502


In [4]:
df.AIRLINE_NAME.replace(to_replace=' Inc.', value='', regex=True, inplace=True)
df.AIRLINE_NAME.replace(to_replace=' Co.', value='', regex=True, inplace=True)
df.AIRLINE_NAME.unique()

array(['United Air Lines', 'Southwest Airlines', 'American Airlines',
       'Comair', 'Mesa Airlines', 'Alaska Airlines',
       'Atlantic Southeast Airlines', 'SkyWest Airlines', 'Endeavor Air',
       'Midwest Airline,', 'Frontier Airlines', 'Delta Air Lines',
       'Spirit Air Lines', 'American Eagle Airlines', 'Hawaiian Airlines',
       'JetBlue Airways', 'Allegiant Air'], dtype=object)

In [25]:
airlines = pd.DataFrame(data=[df.AIRLINE_ID.unique(), df.AIRLINE_NAME.unique()])
airlines = airlines.T
airlines.rename(columns={0: 'AIRLINE_ID', 1: 'AIRLINE_NAME'}, inplace=True)
airlines

,AIRLINE_ID,AIRLINE_NAME
0,19977,United Air Lines
1,19393,Southwest Airlines
2,19805,American Airlines
3,20417,Comair
4,20378,Mesa Airlines
5,19930,Alaska Airlines
6,20366,Atlantic Southeast Airlines
7,20304,SkyWest Airlines
8,20363,Endeavor Air
9,20314,"Midwest Airline,"


In [26]:
airlines.to_csv('2019-airline-delays-and-cancellations/airlines_table.csv', index=False)

In [5]:
airline_historical = pd.DataFrame(df.groupby(['AIRLINE_NAME', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
airline_historical.rename(columns={'DEP_DEL15':'AIRLINE_HISTORICAL'}, inplace=True)
airline_historical

,AIRLINE_NAME,MONTH,AIRLINE_HISTORICAL
0,Alaska Airlines,1,0.189751
1,Alaska Airlines,2,0.130018
2,Alaska Airlines,3,0.155976
3,Alaska Airlines,4,0.131479
4,Alaska Airlines,5,0.144785
...,...,...,...
199,United Air Lines,8,0.223923
200,United Air Lines,9,0.171632
201,United Air Lines,10,0.148566
202,United Air Lines,11,0.138074


In [6]:
airport_historical = pd.DataFrame(df.groupby(['DEP_AIRPORT_ID', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
airport_historical.rename(columns={'DEP_DEL15':'DEP_AIRPORT_HIST'}, inplace=True)
arr_airport_historical = pd.DataFrame(df.groupby(['ARR_AIRPORT_ID', 'MONTH'])['ARR_DEL15'].mean().transpose().reset_index())
arr_airport_historical.rename(columns={'ARR_DEL15':'ARR_AIRPORT_HIST'}, inplace=True)
arr_airport_historical.drop(['MONTH', 'ARR_AIRPORT_ID'], axis=1, inplace=True)
airport_historical = pd.concat([airport_historical, arr_airport_historical], axis=1)
airport_historical

,DEP_AIRPORT_ID,MONTH,DEP_AIRPORT_HIST,ARR_AIRPORT_HIST
0,10140,1,0.109694,0.105850
1,10140,2,0.103746,0.102273
2,10140,3,0.211699,0.170854
3,10140,4,0.158442,0.160950
4,10140,5,0.181122,0.210526
...,...,...,...,...
1147,15919,8,0.253275,0.219008
1148,15919,9,0.140496,0.156000
1149,15919,10,0.200000,0.201465
1150,15919,11,0.120968,0.127119


In [7]:
day_historical = pd.DataFrame(df.groupby(['DAY_OF_WEEK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
day_historical.rename(columns={'DEP_DEL15':'DAY_HISTORICAL'}, inplace=True)
day_historical

,DAY_OF_WEEK,MONTH,DAY_HISTORICAL
0,1,1,0.124096
1,1,2,0.110320
2,1,3,0.160263
3,1,4,0.246152
4,1,5,0.192224
...,...,...,...
79,7,8,0.196210
80,7,9,0.126507
81,7,10,0.163419
82,7,11,0.115448


In [13]:
time_block_hist = pd.DataFrame(df.groupby(['DEP_TIME_BLK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
time_block_hist.rename(columns={'DEP_DEL15':'DEP_BLOCK_HIST'}, inplace=True)
arr_block_hist = pd.DataFrame(df.groupby(['ARR_TIME_BLK', 'MONTH'])['ARR_DEL15'].mean().transpose().reset_index())
arr_block_hist.rename(columns={'ARR_DEL15':'ARR_BLOCK_HIST'}, inplace=True)
arr_block_hist.drop(columns=['MONTH', 'ARR_TIME_BLK'], inplace=True)
time_block_hist = pd.concat([time_block_hist, arr_block_hist], axis=1)
time_block_hist.rename(columns={'DEP_TIME_BLK':'TIME_BLOCK'}, inplace=True)
time_block_hist.dropna(inplace=True)

In [14]:
time_block_hist.MONTH = time_block_hist.MONTH.astype(int)
time_block_hist

,TIME_BLOCK,MONTH,DEP_BLOCK_HIST,ARR_BLOCK_HIST
0,0001-0559,1,0.065665,0.309636
1,0001-0559,2,0.071530,0.342939
2,0001-0559,3,0.065901,0.397368
3,0001-0559,4,0.074822,0.459157
4,0001-0559,5,0.080820,0.529300
...,...,...,...,...
223,2300-2359,8,0.256358,0.356053
224,2300-2359,9,0.148529,0.232694
225,2300-2359,10,0.124332,0.247134
226,2300-2359,11,0.147965,0.209073


In [15]:
airline_historical.to_csv('2019-airline-delays-and-cancellations/airline_historical.csv', index=False)
airport_historical.to_csv('2019-airline-delays-and-cancellations/airport_historical.csv', index=False)
day_historical.to_csv('2019-airline-delays-and-cancellations/day_historical.csv', index=False)
time_block_hist.to_csv('2019-airline-delays-and-cancellations/time_block_historical.csv', index=False)

In [31]:
flight = df.iloc[6]
airports = df.DEP_AIRPORT_ID.unique()
airports.reshape(1, 96).sort()

In [32]:
for i in airports:
    flight[f'DEP_AIRPORT_{i}'] = 0
    
flight

MONTH                   11
DAY_OF_MONTH            23
DAY_OF_WEEK              6
DEP_AIRPORT_ID       12264
ARR_AIRPORT_ID       10529
                     ...  
DEP_AIRPORT_15304        0
DEP_AIRPORT_15370        0
DEP_AIRPORT_15376        0
DEP_AIRPORT_15412        0
DEP_AIRPORT_15919        0
Name: 6, Length: 136, dtype: object

In [3]:
df.DEP_TIME_BLK.unique()

array(['1600-1659', '1300-1359', '1100-1159', '2200-2259', '0700-0759',
       '0800-0859', '1800-1859', '2000-2059', '0900-0959', '1200-1259',
       '1500-1559', '1900-1959', '1400-1459', '1000-1059', '0600-0659',
       '1700-1759', '0001-0559', '2100-2159', '2300-2359'], dtype=object)

In [52]:
a = 5
b = [7, 9, 15]
c = pd.DataFrame(data=[a, b])
c.T

,0,1
0,5,"[7, 9, 15]"
